In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy import signal
from scipy import stats
from scipy import linalg
import time

import sys
sys.path.insert(0, '..')
import specsens as ss

In [1]:
n_bands = 1
band_to_detect = 0
n_fft = 1024
n_stats = 1024 // n_bands

class MonteCarloSim():
    def __init__(self, gens, itrs):
        self.gens = gens  # Number of generations
        self.itrs = itrs  # NUmber of iterations per generation
        self.reset()

    def reset(self):
        self.pfas = list()  # Probability of false alarm list
        self.pds = list()  # Probability of detection list
        self.en_correct_hit_list = list()  # Probability of detection list
        self.en_false_hit_list = list()  # Probability of detection list
        self.en_correct_miss_list = list()  # Probability of detection list
        self.en_false_miss_list = list()  # Probability of detection list
        self.time = None

    def run(self,
            signal_strength=0.,
            noise_strength=0.,
            sample_freq=1e6,
            length=1.,
            pre_pfa=0.1):

        self.reset()

#         n_samples = ss.util.get_signal_length(f_sample=sample_freq,
#                                               t_sec=length)
        n_samples = n_stats
        thr = ss.chi2_stats.get_thr(
            noise_power=noise_strength,
            pfa=pre_pfa,
            n=n_samples,
            dB=True)
        print(f'Threshold is: {thr} with {n_samples} samples')
        
        # Generations loop
        for i in range(self.gens):

            wm = ss.WirelessMicrophone(f_sample=sample_freq, t_sec=length)
            wgn = ss.WhiteGaussianNoise(f_sample=sample_freq, t_sec=length)

            correct_hit = 0
            false_hit = 0
            correct_miss = 0
            false_miss = 0
            en_correct_hit = 0
            en_false_hit = 0
            en_correct_miss = 0
            en_false_miss = 0
            times_signal_present = 0

            # As soon as we introduce noise uncertainty, the detector performs badly
            # This is expected, as energy detectors cant handle noise uncertainty
            # noise_uncertainty = np.random.uniform(-1., 1.)
            noise_uncertainty = 0.
            
            # Iterations loop
            for j in range(self.itrs):
                
                # Generate signal and noise
                sig = wm.get_soft(f_center=4e5, dB=signal_strength)
                noise = wgn.get_signal(dB=noise_strength + noise_uncertainty)
                
                # Randomly decide whether signal should be present
                sig_present = bool(np.random.randint(2))
                if sig_present:
                    both = sig + noise
                    times_signal_present += 1
                else:
                    both = noise

                # Classic energy detector
#                 eng = ss.EnergyDetector.get(both)
                
                # Mutliband stft based energy detector
                sft1 = ss.Stft(
                    n=n_fft,
                    window='flattop')  # currently only working with box window
                f1, psd1 = sft1.stft(sig=both[100:1124],
                                  f_sample=sample_freq,
                                  normalized=False,
                                  dB=False)
                edf1 = ss.FreqEnergyDetector(num_bands=n_bands,
                                            f_sample=1e6,
                                            fft_len=n_fft,
                                            freqs=f1)
                eng1 = edf1.detect(psd1)[band_to_detect]
                
                sft = ss.Stft(
                    n=n_fft,
                    window='box')  # currently only working with box window
                f, psd = sft.stft(sig=both[100:1124],
                                  f_sample=sample_freq,
                                  normalized=False,
                                  dB=False)
                edf = ss.FreqEnergyDetector(num_bands=n_bands,
                                            f_sample=1e6,
                                            fft_len=n_fft,
                                            freqs=f)
                eng = edf.detect(psd)[band_to_detect]
                
#                 plt.figure(figsize=(8, 6))
#                 plt.plot(f, psd)
#                 plt.show()
                    
                # Threshold
                sig_detected1 = eng1 > thr
                sig_detected = eng > thr
                
                if sig_detected1 != sig_detected:
                    print(f'Flattop {sig_detected1} {eng1}  Box {sig_detected} {eng}  sig {sig_present}')
                
                # Log signal and detection outcome
                if sig_present and sig_detected1:
                    correct_hit += 1
                    en_correct_hit += eng/eng1
                elif sig_present and not sig_detected1:
                    false_miss += 1
                    en_false_miss += eng/eng1
                elif not sig_present and sig_detected1:
                    false_hit += 1
                    en_false_hit += eng/eng1
                else:
                    correct_miss += 1
                    en_correct_miss += eng/eng1
            
            # Compute stats and store in list
            pfa_tmp = false_hit / (self.itrs - times_signal_present)
            pd_tmp = correct_hit / times_signal_present
            self.pfas.append(pfa_tmp)
            self.pds.append(pd_tmp)
            
            self.en_correct_hit_list.append(en_correct_hit / correct_hit)
            self.en_false_miss_list.append(en_false_miss / false_miss)
            self.en_false_hit_list.append(en_false_hit / false_hit)
            self.en_correct_miss_list.append(en_correct_miss / correct_miss)
            
            # Print simulation progress
            rem, per = self.runtime_stats(self.gens, i)
            print('%6.2fs left at %5.2f%%' % (rem, per))
        
        # Compute stats from list 
        pfa = np.sum(self.pfas) / self.gens
        pd = np.sum(self.pds) / self.gens
        
        print(f'1: {np.sum(self.en_correct_hit_list) / self.gens}')
        print(f'2: {np.sum(self.en_false_miss_list) / self.gens}')
        print(f'3: {np.sum(self.en_false_hit_list) / self.gens}')
        print(f'4: {np.sum(self.en_correct_miss_list) / self.gens}')

        return pfa, pd

    def runtime_stats(self, total_itr, current_itr):
        if self.time is None: # First iteration cant predict time
            self.time = time.time()
            return float('inf'), 0.0
        delta_time = time.time() - self.time
        self.time = time.time()
        remaining_itr = total_itr - current_itr
        remaining_time = delta_time * remaining_itr
        percent_done = current_itr / total_itr * 100.0
        return remaining_time, percent_done

    def print_convergence(self):
        plt.figure(figsize=(8, 6))
        plt.grid(linewidth=0.3)
        for i in range(self.gens):
            inter = np.sum(self.pfas[0:i]) / i
            plt.plot(i, inter, 'kx')
        plt.show()


signal_strength = 12.
noise_strength = 23.
theo_pfa = 0.1
length = 0.002
sample_freq = 1e6

n_samples = n_stats
thr = ss.chi2_stats.get_thr(
    noise_power=noise_strength,
    pfa=theo_pfa,
    n=n_samples,
    dB=True)
theo_pd = ss.chi2_stats.get_pd(noise_strength, signal_strength, thr, n_samples, dB=True, bands=n_bands)
print(f'Theory     pfa {theo_pfa}')
print(f'Theory     pd  {theo_pd}')

sim = MonteCarloSim(100, 300)
pfa, pd = sim.run(signal_strength=signal_strength,
                  noise_strength=noise_strength,
                  sample_freq=sample_freq,
                  length=0.002,
                  pre_pfa=theo_pfa)

sim.print_convergence()

print(f'Theory     pfa {theo_pfa}')
print(f'Simulation pfa {pfa}')

n_samples = n_stats
thr = ss.chi2_stats.get_thr(
    noise_power=noise_strength,
    pfa=theo_pfa,
    n=n_samples,
    dB=True)
theo_pd = ss.chi2_stats.get_pd(noise_strength, signal_strength, thr, n_samples, dB=True, bands=n_bands)
print(f'Theory     pd  {theo_pd}')
print(f'Simulation pd  {pd}')
print(f'Threshold      {thr}')

NameError: name 'ss' is not defined

In [13]:
signal_fac = ss.util.dB_to_factor_power(signal_strength)
noise_fac  = ss.util.dB_to_factor_power(noise_strength) / n_bands
n_samples = n_stats
thr = ss.chi2_stats.get_thr(
    noise_power=noise_fac,
    pfa=theo_pfa,
    n=n_samples,
    dB=False)
theo_pd = ss.chi2_stats.get_pd(noise_fac, signal_fac, thr, n_samples, dB=False, bands=1)
print(f'Theory     pd  {theo_pd}')
print(f'Simulation pd  {pd}')
print(f'Threshold      {thr}')

Theory     pd  0.7769212351685021
Simulation pd  0.10054659366376216
Threshold      1621.6613792759565
